In [1]:
# for vectorized operations
import numpy as np

# for dataframe manipulation
import pandas as pd

# for vizualizations
import matplotlib.pyplot as plt
import seaborn as sns

# for statistical calculations
import scipy.stats as stats

# for obtaining stock datasets
from pydataset import data

# for manipulation of time data
from datetime import date

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import sklearn as sk

# filter out warnings
import warnings
warnings.filterwarnings('ignore')

# our own functions for accessing our sql database
from env import get_db_url, user, password, host

# our own scripts
import acquire
import prepare
import explore

# pandas display preferences
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
#pd.option_context('display.max_rows', None)


### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |

In [2]:
positive_case = 'dog'

TP = 46
FP = 13
TN = 34
FN = 7

total_observations = TP + FP + TN + FN

actual_positives = TP + FN

predicted_positives = TP + FP

correct_predictions = TP + TN

In [3]:
model_accuracy = correct_predictions / total_observations
model_accuracy

0.8

In [4]:
model_recall = TP / actual_positives
model_recall

0.8679245283018868

In [5]:
model_precision = TP / predicted_positives
model_precision

0.7796610169491526

In [6]:
# baseline will always predict positive (most common observation)

In [7]:
baseline_accuracy = actual_positives / total_observations
baseline_accuracy

0.53

In [8]:
baseline_recall = actual_positives / actual_positives
baseline_recall

1.0

In [9]:
baseline_precision =  TP / total_observations
baseline_precision

0.46

#### In the context of this problem, what is a false positive?

    a false positive is predicting that the animal is a dog when in actuality it is a cat.
    
#### In the context of this problem, what is a false negative?

    a false negative is  predicting that the animal is a cat when in actuality it is a dog

#### How would you describe this model?

    This model's accuracy of 80% is 34% better than baseline. 
    The model's precision of 78% is 32% than baseline. 
    The model's recall is 87%. 


### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

#### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

#### Use the predictions dataset and pandas to help answer the following questions:

In [10]:
df = pd.read_csv('c3.csv')

#### 3a. An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [11]:
# recall is the most appropriate evaluation metric
# Model 3 has the best recall (see below)

In [12]:
df.head(3)

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect


In [13]:
model_1 = df[['actual', 'model1']].rename(columns={'model1': 'prediction'})
model_1['baseline'] = model_1.actual.mode()[0]

model_2 = df[['actual', 'model2']].rename(columns={'model2': 'prediction'})
model_2['baseline'] = model_2.actual.mode()[0]


model_3 = df[['actual', 'model3']].rename(columns={'model3': 'prediction'})
model_3['baseline'] = model_3.actual.mode()[0]

In [14]:
def evaluate_classifier(df, positive):

    # accuracy -- overall hit rate
    model_accuracy = (df.prediction == df.actual).mean()
    baseline_accuracy = (df.baseline == df.actual).mean()

    # precision -- how good are our positive predictions?
    # precision -- model performance | predicted positive
    subset = df[df.prediction == positive]
    model_precision = (subset.prediction == subset.actual).mean()
    subset = df[df.baseline == positive]
    baseline_precision = (subset.baseline == subset.actual).mean()

    # recall -- how good are we at detecting actual positives?
    # recall -- model performance | actual positive
    subset = df[df.actual == positive]
    model_recall = (subset.prediction == subset.actual).mean()
    baseline_recall = (subset.baseline == subset.actual).mean()

    print(f'   model accuracy: {model_accuracy:.2%}')
    print(f'baseline accuracy: {baseline_accuracy:.2%}')
    print()
    print(f'   model recall: {model_recall:.2%}')
    print(f'baseline recall: {baseline_recall:.2%}')
    print()
    print(f'model precision: {model_precision:.2%}')
    print(f'baseline precision: {baseline_precision:.2%}')



In [15]:
models = [model_1, model_2, model_3]
positive = 'Defect'

line_break = ('\n' + '-' * 30 + '\n')

for i, model in enumerate(models):
    
    model_number = i + 1
    
    print(f'Model #{model_number}:\n')
    evaluate_classifier(model, positive)
    print(line_break)

Model #1:

   model accuracy: 95.00%
baseline accuracy: 92.00%

   model recall: 50.00%
baseline recall: 0.00%

model precision: 80.00%
baseline precision: nan%

------------------------------

Model #2:

   model accuracy: 56.00%
baseline accuracy: 92.00%

   model recall: 56.25%
baseline recall: 0.00%

model precision: 10.00%
baseline precision: nan%

------------------------------

Model #3:

   model accuracy: 55.50%
baseline accuracy: 92.00%

   model recall: 81.25%
baseline recall: 0.00%

model precision: 13.13%
baseline precision: nan%

------------------------------



#### 3b. Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you they really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [16]:
# precision is the most appropriate metric
# Model 1 is the best fit for this use case (see above)

#### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

#### At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

#### Several models have already been developed with the data, and you can find their results here. Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [17]:
df = pd.read_csv('gives_you_paws.csv')
df.head(3)

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog


In [18]:
df['baseline'] = df.actual.mode()[0]

model_1 = df[['actual', 'model1', 'baseline']].rename(columns={'model1': 'prediction'})

model_2 = df[['actual', 'model2', 'baseline']].rename(columns={'model2': 'prediction'})

model_3 = df[['actual', 'model3', 'baseline']].rename(columns={'model3': 'prediction'})

model_4 = df[['actual', 'model4', 'baseline']].rename(columns={'model4': 'prediction'})

#### 4a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [19]:
models = [model_1, model_2, model_3, model_4]
positive = 'dog'

line_break = ('\n' + '-' * 30 + '\n')

for i, model in enumerate(models):
    
    model_number = i + 1
    
    print(f'Model #{model_number}:\n')
    evaluate_classifier(model, positive)
    print(line_break)

Model #1:

   model accuracy: 80.74%
baseline accuracy: 65.08%

   model recall: 80.33%
baseline recall: 100.00%

model precision: 89.00%
baseline precision: 65.08%

------------------------------

Model #2:

   model accuracy: 63.04%
baseline accuracy: 65.08%

   model recall: 49.08%
baseline recall: 100.00%

model precision: 89.32%
baseline precision: 65.08%

------------------------------

Model #3:

   model accuracy: 50.96%
baseline accuracy: 65.08%

   model recall: 50.86%
baseline recall: 100.00%

model precision: 65.99%
baseline precision: 65.08%

------------------------------

Model #4:

   model accuracy: 74.26%
baseline accuracy: 65.08%

   model recall: 95.57%
baseline recall: 100.00%

model precision: 73.12%
baseline precision: 65.08%

------------------------------



Model 1's accuracy is 14% better than baseline

Model 2's accuracy is 2% lower than baseline

Model 3's accuracy is 14% lower than baseline

Model 4's accuracy is 9% higher than baseline

#### 4b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recomend for Phase I? For Phase II?

In [20]:
# i don't understand what is meant by "a team that solely deals with dog pictures"

# does this mean they only receive dog pictures as input? - this wouldn't make sense - how would they know ahead 
# of time which pictures were dogs?

# does this mean that once dogs are identified in phase I, they are sent to this team for further analysis
# in phase II?

#### 5. Apply the following functions to the data from the previous problem:


    sklearn.metrics.accuracy_score
    sklearn.metrics.precision_score
    sklearn.metrics.recall_score
    sklearn.metrics.classification_report


In [21]:
df.head(3)

,actual,model1,model2,model3,model4,baseline
0,cat,cat,dog,cat,dog,dog
1,dog,dog,cat,cat,dog,dog
2,dog,cat,cat,cat,dog,dog


In [22]:
positive_label = 'dog'

for col in df.drop(columns='actual'):
    print(col)
    print(f'\tAccuracy:\t{sk.metrics.accuracy_score(df.actual, df[col]):.4f}')
    print(f'\tPrecision:\t{sk.metrics.precision_score(df.actual, df[col], pos_label=positive_label):.4f}')
    print(f'\tRecall:\t\t{sk.metrics.recall_score(df.actual, df[col], pos_label=positive_label):.4f}')


model1
	Accuracy:	0.8074
	Precision:	0.8900
	Recall:		0.8033
model2
	Accuracy:	0.6304
	Precision:	0.8932
	Recall:		0.4908
model3
	Accuracy:	0.5096
	Precision:	0.6599
	Recall:		0.5086
model4
	Accuracy:	0.7426
	Precision:	0.7312
	Recall:		0.9557
baseline
	Accuracy:	0.6508
	Precision:	0.6508
	Recall:		1.0000


In [23]:
line_break = ('-' * 55)

for col in df.drop(columns='actual'):
    print(line_break)
    print(col.upper())
    print(sk.metrics.classification_report(df.actual, df.model1))

-------------------------------------------------------
MODEL1
              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84      3254

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000

-------------------------------------------------------
MODEL2
              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84      3254

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000

-------------------------------------------------------
MODEL3
              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84     